In [1]:
packages = {
    "requests": "requests",
    "urllib": None,  # Part of the standard library, no need to install
    "types": None,  # Part of the standard library, often no need to install additional packages
    "pandas": "pandas",
    "fiona": "fiona",
    "json": None,  # Part of the standard library, no need to install
    "numpy": "numpy",
    "geopandas": "geopandas",
    "matplotlib": "matplotlib",
    "pandas": "pandas",
    "tifffile": "tifffile",
    "pyproj": "pyproj",
    "rasterio": "rasterio",
    "aiohttp": "aiohttp",
    "os": None  # Part of the standard library, no need to install
}

def install_package(name):
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", name])

for package, install_name in packages.items():
    if install_name is not None:
        try:
            __import__(package)
        except ImportError:
            print(f"Package {package} not found, installing {install_name}...")
            install_package(install_name)
    else:
        print(f"Package {package} is part of the standard library or not installable via pip.")
        
import requests
import urllib
import concurrent.futures
import types
import pandas as pd
import pickle
import json
import numpy as np
import rasterio
import concurrent.futures
import time
import src.StreetViewData
from threading import Lock
from pyproj import Transformer
import matplotlib.pyplot as plt
from multiprocessing import Pool
import os

Package urllib is part of the standard library or not installable via pip.
Package types is part of the standard library or not installable via pip.
Package json is part of the standard library or not installable via pip.
Package os is part of the standard library or not installable via pip.


In [2]:
folder_dir = "images"
meta_base = 'https://maps.googleapis.com/maps/api/streetview/metadata?'
pic_base = 'https://maps.googleapis.com/maps/api/streetview?'

# Insert API Key here

api_key = 'API_KEY'
meta_params = {'key': api_key,
               'location': '36.852833,-121.782302'}
pic_params = {'key': api_key,
              'location': '36.852833,-121.782302',
              'size': "640x640",
              'souce': 'outdoors'}
seen_images = set()

In [3]:
# Verify the APi works on a good case

meta = requests.get(meta_base, meta_params)
meta.json()

{'copyright': '© Google',
 'date': '2021-05',
 'location': {'lat': 36.85277500085382, 'lng': -121.7823315385135},
 'pano_id': 'LmAli5-M7Jt4S548oBPwvQ',
 'status': 'OK'}

In [4]:
# Make sure I don't request for duplicate images

def load_seen(folder):
  seen_list = os.listdir('../{}'.format(folder))
  for image_name in seen_list:
    loc = image_name.split('_')[0]
    seen_images.add(loc)
load_seen('images')

In [5]:
# Date Filter

def image_is_in_season(date):
  year, month = map(int, date.split('-'))
  return (year == 2018 and month <= 12 and month >= 9)

In [6]:
# Reference the metadata to see if the location is good

def verify_meta(meta) -> bool:
    if meta.status_code == 200:
        response_data = meta.json()  # Parse the JSON response
        if response_data.get("status") == "OK":
            return True
        else:
            return False
    else:
        return False

In [7]:
# Load Geo Data
# poi = fiona.open("../data/geos/wheatfield_shapefields.shp")

# Load Raster Data
# poi = rasterio.open("../data/rasters/MaskedCropsExport.tif")

In [8]:
# Load globals and close file

dataset = rasterio.open('../data/rasters/MaskedCropsExport.tif')
indeces = np.where(dataset.read(1) == 3)

crs = dataset.crs
transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
transform = dataset.transform

dataset.close()

In [ ]:
# Define value for threading
CONNECTIONS = 50
TIMEOUT = 5

# Define globals
output_file = "succeeded_coordinates.txt"
succeeded = set()
count = 0
lock = Lock()

def write_to_file(lat, lng):
    with open(output_file, "a") as file:
        file.write(f"{lat},{lng}\n")

def load_url(params):
    global count
    meta_response = requests.get(meta_base, params=params)
    if verify_meta(meta_response):
        resp = meta_response.json()
        if image_is_in_season(resp['date']):
            lat, lng = (resp['location']['lat'], resp['location']['lng'])
            with lock:
                if (lat, lng) not in succeeded:
                    count += 1
                    succeeded.add((lat, lng))
                    print(f'Succeeded {count}')
                    write_to_file(lat, lng)
            return (lat, lng)
    return None

def run_wheatfield_finder():
    
    time1 = time.time()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
        
        for row, col in zip(indeces[0], indeces[1]):
            x, y = transform * (col, row)
            long, lat = transformer.transform(x, y)
            loc = "{},{}".format(lat, long)
            curr_params = {'key': api_key, 'location': loc}
            _ = executor.submit(load_url, curr_params)
        
    time2 = time.time()

    print(f'Took {time2-time1:.2f} s')
    print(f'Processed {len(succeeded)} items.')
    
run_wheatfield_finder()

In [ ]:
# Cache the results to we dont have to worry about losing the session
file_path = 'succeeded2.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(succeeded, file)

print("Set saved successfully to", file_path)

In [26]:
# Wrapper to make a request. Takes a location ("lat,lon") and bearing (angle), 

def load_streetview(location, bearing=None):

  if (location in seen_images):
    return False

  meta_params = {'key': api_key,
               'location': location}

  if bearing is not None:
    pic_params = {'key': api_key,
                  'location': location,
                  'size': "640x640",
                  'heading': bearing}
  else:
    pic_params = {'key': api_key,
                  'location': location,
                  'size': "640x640"}
  meta_response = requests.get(meta_base, params=meta_params)
  if (verify_meta(meta_response) == False):
    return False
  date = meta_response.json()['date']

  if (not image_is_in_season(date)):
    return False

  pano_id = meta_response.json()['pano_id']
  pic_response = requests.get(pic_base, params=pic_params)

  pic = StreetViewImage(pic_response, location, bearing, date, pano_id)
  pic.save_to_file(folder_dir)
  
  return True

In [35]:
# For each viable location, make a request for the image. It will automatically route to the correct folder

fail_count = 0
bearings = []
for loc in succeeded:
  print(str(loc[0]) + "," + str(loc[1]))
  res = load_streetview(str(loc[0]) + "," + str(loc[1]), 0)
  if (res == False):
    fail_count += 1
    print(fail_count)

57.09021438354,9.68961969413625
56.58633670519406,10.18338976599919
56.29576478567706,10.51256302815402
56.77377733803417,10.09575370219504
57.46776741318843,10.46838355136887
57.08826473745475,9.68975363902504
57.08509306557903,9.689886219090285
55.4352423,12.0503643
56.9940418243273,8.793626668439625
57.46627062886132,10.46813240663957
56.96961777756042,9.964977590565224
56.29550330357976,10.51347636627355
57.00539904222405,8.92911485943123
56.60912210532733,10.19254233171912
56.90377626499565,9.225489447653015
57.17519403493598,10.10988266891233
56.77319002820914,10.09369594871309
56.77433053956392,10.09840089943665
56.33716946282153,10.41150325007182
56.94299893906916,10.00640030414075
56.94642106946563,10.02126998900725
57.46615994382057,10.4681009959445
56.99426864838183,8.79412535459956
56.77409299319635,10.09700539346028
56.77421308554577,10.09761749233185
55.209673956247,9.6355746409885
57.46704760991851,10.46826995731223
57.00019035952798,8.909407442271732
56.29407008954148,1